In [ ]:
/**
 https://www.bambooweekly.com/government-corruption/ 
 https://www.bambooweekly.com/government-corruption-436/ 
 https://github.com/JoergEm/Bamboo-Weekly/tree/main
*/

In [ ]:
// Imports
import * as aq from "npm:arquero";
import * as math from "npm:mathjs";
import * as XLSX from "npm:xlsx";
import { join } from "jsr:@std/path";
import * as fs from "jsr:@std/fs";
console.log("Imports ✅");

In [ ]:
// Function creating local folders
async function createFolders(folders: string[]): Promise<boolean> {
  try {
    for (const folder of folders) {
      const folderPath = join(Deno.cwd(), folder);
      try {
        await Deno.stat(folderPath);
      } catch {
        await Deno.mkdir(folderPath, { recursive: true });
      }
    }
  } catch (error) {
    console.error("Error ❌", error);
    return false;
  }

  console.log("Folders ✅");
  return true;
}

In [ ]:
// Function downloading data locally
async function downloadData(url: string, filename: string): Promise<boolean> {
  try {
    const response = await fetch(url);

    if (!response.ok) {
      console.error(`HTTP Error: ${response.status} ${response.statusText}`);
      return false;
    }

    const arrayBuffer = await response.arrayBuffer();
    const data = new Uint8Array(arrayBuffer);
    await Deno.writeFile(filename, data);
    console.log(`Downloaded ✅: ${filename}`);
    return true;

  } catch (error) {
    console.error("Error ❌", error);
    return false;
  }
}

In [ ]:
// Links and folders to recieve data and read into DataFrame
const url = "https://images.transparencycdn.org/images/CPI2022_GlobalResultsTrends.xlsx";
const filename = "CPI2022_GlobalResultsTrends.xlsx";
const folders = ["data", "results"];
const filepath = join(folders[0], filename);

await createFolders(folders);
async function exists(path: string): Promise<boolean> {
  try {
    await Deno.stat(path);
    return true;
  } catch {
    return false;
  }
}

if (!(await exists(filepath))) {
  const success = await downloadData(url, filepath);
  if (success) {
    console.log("Data ✅ downloaded");
  } else {
    console.log("Error ❌ downloading data");
  }
} else {
  console.log("Data already exists ✅");
}

let data: aq.Table | null = null;
if (await exists(filepath)) {
  const arrayBuffer = await Deno.readFile(filepath);
  const workbook = XLSX.read(arrayBuffer, { type: "array" });
  const sheet = workbook.Sheets["CPI 2022 (final)"];
  const sheetJson = XLSX.utils.sheet_to_json(sheet, { range: 2 });
  data = aq.from(sheetJson);

  console.log("Data loaded into DataFrame ✅");
}

console.log(`Filepath: ${filepath}`);

In [ ]:
// According to Transparency International, what five countries were least corrupt in 2022?
if (data) {
  const sorted = data.orderby("Rank");
  const top5 = aq.from(sorted.objects().slice(0, 5));
  const top5Countries = top5.select("Country / Territory");
  const countriesArray = top5Countries.array("Country / Territory");

  console.log(countriesArray);
}


In [ ]:
// According to the same data, what five countries were most corrupt in 2022?
if (data) {
  const sorted = data.orderby("Rank");
  const last5 = aq.from(sorted.objects().slice(-5));
  const last5Countries = last5.select("Country / Territory");
  const countriesArray = last5Countries.array("Country / Territory");

  console.log(countriesArray);
}
